In [1]:
!pip install -r requirements.txt

In [2]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import shapely

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

from utilities import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
bus_hqtc = gpd.read_parquet(f'{GCS_FILE_PATH}shape_hqta_dissolve.parquet')
bus_hqtc = bus_hqtc[bus_hqtc['hq_transit_corr']]
bus_hqtc['hqta_type'] = 'hq_transit_corr'

In [4]:
rail_ferry_brt_stops = gpd.read_parquet(f'{GCS_FILE_PATH}rail_brt_ferry.parquet')
rail_ferry_brt_stops['hqta_type'] = 'major_transit_stop'

In [7]:
rail_ferry_brt_stops.columns

Index(['stop_id', 'calitp_itp_id', 'stop_lat', 'stop_lon', 'itp_id_stop_id',
       'itp_id_trip_id', 'route_id', 'itp_id_route_id', 'trip_id', 'geometry',
       'calitp_url_number', 'route_type', 'agency_id', 'route_short_name',
       'route_long_name', 'route_desc', 'route_url', 'route_color',
       'route_text_color', 'route_sort_order', 'continuous_pickup',
       'continuous_drop_off', 'calitp_extracted_at', 'hqta_type'],
      dtype='object')

In [10]:
# geoparquet_gcs_export(bus_hqtc, 'bus_hqtc')

In [11]:
# geoparquet_gcs_export(rail_ferry_brt_stops, 'rail_ferry_brt_stops')

### High Quality Transit Areas Relevant Statutes

[PRC 21155](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21155.&lawCode=PRC)
* _(3) be within one-half mile of a major transit stop or high-quality transit corridor included in a regional transportation plan._
* Major transit stop definition: _A major transit stop is as defined in Section 21064.3, except that, for purposes of this section, it also includes major transit stops that are included in the applicable regional transportation plan_
* High-quality transit corridor definition: _For purposes of this section, a high-quality transit corridor means a corridor with fixed route bus service with service intervals no longer than 15 minutes during peak commute hours._
    * Unable to locate definition of "peak commute hours"

[PRC 21064.3](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?sectionNum=21064.3.&lawCode=PRC)
* _Major transit stop means a site containing any of the following:
(a) An existing rail or bus rapid transit station.
(b) A ferry terminal served by either a bus or rail transit service.
(c) The intersection of two or more major bus routes with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods._
    * "Intersection" may not be sufficiently well-defined for this analysis

[PRC 21060.2](https://leginfo.legislature.ca.gov/faces/codes_displaySection.xhtml?lawCode=PRC&sectionNum=21060.2.&highlight=true&keyword=bus%20rapid%20transit)
* _(a) “Bus rapid transit” means a public mass transit service provided by a public agency or by a public-private partnership that includes all of the following features:
(1) Full-time dedicated bus lanes or operation in a separate right-of-way dedicated for public transportation with a frequency of service interval of 15 minutes or less during the morning and afternoon peak commute periods.
(2) Transit signal priority.
(3) All-door boarding.
(4) Fare collection system that promotes efficiency.
(5) Defined stations._
    * Unlikely to determine if a service qualifies as BRT under this definition using GTFS alone

## Bus Major Stops

In [8]:
bus_hqtc.head(3)

,hq_transit_corr,shape_id,geometry,calitp_itp_id,hqta_segment_id,segment_sequence,stop_id,am_max_trips,pm_max_trips,hqta_type
2256,True,01582b61-5a02-4e6c-bd37-bad46bb036a9,"POLYGON ((-133311.510 166655.910, -133353.419 ...",48,1937384376,0,12f4ecb6-c161-480e-8ec2-d7e3b63c7e38,7.0,8.0,hq_transit_corr
2257,True,0500,"POLYGON ((212373.929 -486798.063, 212402.264 -...",142,4126984225,1,4654,5.0,7.0,hq_transit_corr
2258,True,0843,"MULTIPOLYGON (((200815.723 -472198.343, 200811...",142,1555422069,0,6679,5.0,6.0,hq_transit_corr


In [9]:
gdf = bus_hqtc
output = gpd.GeoDataFrame()

def find_intersections(row):
    global output
    # display(row)
    # display(row.shape_id)
    this_row = gdf >> filter(_.shape_id == row.shape_id)
    not_this_row = gdf >> filter(_.shape_id != row.shape_id)
    # print(type(filtered))
    clip_row = gpd.clip(this_row, not_this_row)
    output = output.append(clip_row)
    return

In [10]:
# _test = bus_hqtc.apply(find_intersections, axis=1)

In [11]:
# output.geometry = output.geometry.buffer(50)

In [12]:
# geoparquet_gcs_export(output, 'major_bus_stops_working')

In [10]:
major_bus = gpd.read_parquet(f'{GCS_FILE_PATH}major_bus_stops_working.parquet')

In [11]:
def drop_big_areas(geometry):
    if type(geometry) == shapely.geometry.multipolygon.MultiPolygon:
        filtered = [x for x in list(geometry.geoms) if x.length < 1000]
        if len(filtered) > 0:
            return shapely.geometry.MultiPolygon(filtered)
    elif type(geometry) == shapely.geometry.polygon.Polygon:
        if geometry.length < 1000:
            return geometry
    else:
        return np.nan

In [12]:
major_bus['geometry'] = major_bus.geometry.apply(drop_big_areas)

In [28]:
row_per_stop = gpd.GeoDataFrame()
def explode_geoms(row):
    global row_per_stop
    if type(row.geometry) == shapely.geometry.multipolygon.MultiPolygon:
        for geom in row.geometry.geoms:
            row.geometry = geom.centroid
            row_per_stop = row_per_stop.append(row)
    elif type(row.geometry) == shapely.geometry.polygon.Polygon:
        row.geometry = row.geometry.centroid
        row_per_stop = row_per_stop.append(row)
    
    return row

In [29]:
major_bus = major_bus.dropna(subset=['geometry'])

In [30]:
major_bus.apply(explode_geoms, axis=1)

,hq_transit_corr,shape_id,geometry,level_0,calitp_itp_id,hqta_segment_id,n_trips,segment_sequence,stop_id,am_peak,pm_peak,hqta_type
2464,True,0500,POINT (193061.249 -465491.440),0,142,2197551287,134.0,0,3977,9.0,8.0,hq_transit_corr
2467,True,100706_SEPT21,POINT (149243.534 -435856.085),0,182,2368710740,120.0,1,16374,6.0,8.0,hq_transit_corr
2468,True,100708_SEPT21,POINT (159441.767 -444289.585),3,182,2883785450,72.0,3,9785,4.7,4.7,hq_transit_corr
2471,True,1020067_SEPT21,POINT (150511.825 -447354.169),3,182,2238923216,115.0,4,11283,8.0,8.0,hq_transit_corr
2475,True,103717,POINT (-178950.630 -71003.248),1,294,2280977661,136.0,3,480,7.0,8.0,hq_transit_corr
...,...,...,...,...,...,...,...,...,...,...,...,...
3231,True,shp-73-01,POINT (-194726.607 -31504.765),0,4,1652690359,220.0,0,3708,9.3,11.7,hq_transit_corr
3232,True,shp-73-51,POINT (-194726.607 -31504.765),0,4,1682558812,238.0,0,2712,13.3,10.0,hq_transit_corr
3236,True,shp-88-07,POINT (-200443.266 -13810.682),1,4,3310950755,83.0,1,2026,4.7,5.0,hq_transit_corr
3238,True,shp-97-57,POINT (-187275.985 -33268.849),0,4,4254707562,80.0,0,4320,4.3,4.7,hq_transit_corr


In [31]:
row_per_stop = row_per_stop.reset_index(drop=True)

In [32]:
row_per_stop = row_per_stop[['calitp_itp_id', 'stop_id', 'geometry']]

In [33]:
row_per_stop['hqta_type'] = 'major_transit_stop'

In [34]:
row_per_stop = row_per_stop.set_crs('EPSG:6414')

In [35]:
row_per_stop.head(3)

,calitp_itp_id,stop_id,geometry,hqta_type
0,142,3977,POINT (199058.382 -471125.032),major_transit_stop
1,142,3977,POINT (198531.592 -471368.189),major_transit_stop
2,142,3977,POINT (193068.693 -465847.391),major_transit_stop


In [36]:
geoparquet_gcs_export(row_per_stop, 'major_bus_stops')

/home/jovyan/data-analyses/high_quality_transit_areas/utilities.py:85: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  gdf.to_parquet(f"{name}.parquet")


In [24]:
# major_bus_operators = [int(x) for x in list(major_bus.calitp_itp_id.astype(int).unique())]

In [25]:
# tbl_stops = (tbl.gtfs_schedule.stops() 
#          >> filter(_.calitp_itp_id.isin(major_bus_operators))
#          >> select(_.stop_id, _.stop_lat, _.stop_lon, _.calitp_itp_id)
#          >> collect()
#             )
# tbl_stops = gpd.GeoDataFrame(tbl_stops,
#                  geometry = gpd.points_from_xy(tbl_stops.stop_lon, tbl_stops.stop_lat),
#                  crs = 'EPSG:4326').to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)

In [26]:
# def join_stops(df):
#     stops_filtered = tbl_stops >> filter(_.calitp_itp_id == int(df.calitp_itp_id.iloc[0]))
#     df = row_per_stop.sjoin(stops_filtered, how='inner', predicate='contains')
#     return df

In [27]:
# join_to_stop = row_per_stop.groupby('calitp_itp_id').apply(join_stops)

#### Definitions and Output:

* hqta_type: major_transit_stop
* stop_id: one stop id... (not ideal, but oh well)
* geometry: .buffer(700)?
* _can alway pull more info spatially_

### Combining and Buffering

* General buffer distance: 1/2mi ~= 805 meters
* Bus corridors are already buffered 50 meters, so will buffer 755 meters

In [28]:
bus_hqtc.geometry = bus_hqtc.geometry.buffer(755)
row_per_stop.geometry = row_per_stop.geometry.buffer(805)
rail_ferry_brt_stops.geometry = rail_ferry_brt_stops.geometry.buffer(805)

In [29]:
all_hqta = (bus_hqtc
            .append(rail_ferry_brt_stops)
            .append(row_per_stop)
            .fillna('')
           )

In [30]:
all_hqta = all_hqta[['calitp_itp_id', 'geometry', 'hqta_type']]
all_hqta = all_hqta.reset_index(drop=True)
all_hqta['calitp_itp_id'] = all_hqta['calitp_itp_id'].astype('int64')

### Format for export

In [31]:
agency_names = (tbl.gtfs_schedule.agency()
 >> select(_.calitp_itp_id, _.agency_name)
 >> collect()
)

In [32]:
all_hqta = all_hqta >> inner_join(_, agency_names, on='calitp_itp_id')

In [6]:
all_hqta = gpd.read_parquet(f'{GCS_FILE_PATH}ca_high_quality_transit.parquet')

In [16]:
metro = bus_hqtc >> filter(_.calitp_itp_id == '182')

In [18]:
map_hqta(metro, 'pm_max_trips')

Map(center=[34.078792878404975, -118.30956119336547], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [ ]:
geoparquet_gcs_export(all_hqta, 'ca_high_quality_transit')
# all_hqta.to_file('./ca_high_quality_transit.geojson', driver='GeoJSON')
# all_hqta.to_file('./ca_high_quality_transit/ca_high_quality_transit.shp')

## Static Map Images (all HQTAs)

![bay](img/bay_valley_all.png)

![fresno](img/fres_all.png)

![san_diego](img/sd_all.png)